# Analyse des Stratégies de Schedulers et Influence d'Epsilon

Ce notebook analyse les différentes stratégies de schedulers pour l'apprentissage robuste aux attaques adversaires et évalue l'influence du paramètre epsilon sur ces stratégies.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from Schedulers import (
    ConstantScheduler,
    LinearScheduler,
    LinearUniformMixScheduler,
    ExponentialScheduler,
    CyclicScheduler,
    RandomScheduler,
    CompositeScheduler
)
import pandas as pd
from tqdm import tqdm
import seaborn as sns

# Configuration
K_MIN = 0
K_MAX = 10
MAX_EPOCHS = 100

# Style des plots
plt.style.use('seaborn')
sns.set_palette('husl')

## 1. Initialisation des Schedulers

Nous initialisons tous les schedulers de base avec leurs paramètres par défaut.

In [ ]:
# Initialisation des schedulers de base
base_schedulers = {
    'Constant': ConstantScheduler(K_MIN, K_MAX),
    'Linear': LinearScheduler(K_MIN, K_MAX),
    'LinearUniformMix': LinearUniformMixScheduler(K_MIN, K_MAX),
    'Exponential': ExponentialScheduler(K_MIN, K_MAX),
    'Cyclic': CyclicScheduler(K_MIN, K_MAX),
    'Random': RandomScheduler(K_MIN, K_MAX)
}

# Configuration des stratégies d'epsilon
epsilon_configs = {
    'constant_small': 0.1,
    'constant_medium': 0.3,
    'constant_large': 0.5,
    'linear': 'linear',
    'cyclic': 'cyclic',
    'random': 'random'
}

## 2. Collecte des Données des Schedulers

In [ ]:
def collect_scheduler_data():
    data = []
    for scheduler_name, scheduler in tqdm(base_schedulers.items(), desc="Collecting data"):
        for epoch in range(MAX_EPOCHS):
            eps, k_dist = scheduler(epoch, MAX_EPOCHS)
            for k, prob in k_dist.items():
                data.append({
                    'epoch': epoch,
                    'scheduler': scheduler_name,
                    'k': k,
                    'probability': prob,
                    'epsilon': eps
                })
    return pd.DataFrame(data)

# Collecte des données
base_df = collect_scheduler_data()
base_df.head()

## 3. Visualisation des Stratégies

In [ ]:
plt.figure(figsize=(15, 10))
for scheduler_name in base_schedulers.keys():
    scheduler_data = base_df[base_df['scheduler'] == scheduler_name]
    plt.plot(scheduler_data['epoch'], scheduler_data['k'], 
            label=scheduler_name, alpha=0.7)

plt.title('Comparison of Different Scheduler Strategies')
plt.xlabel('Epoch')
plt.ylabel('K Value')
plt.legend()
plt.grid(True)
plt.show()

## 4. Calcul des Métriques des Schedulers

In [ ]:
def calculate_scheduler_metrics(df):
    metrics = []
    for scheduler_name in base_schedulers.keys():
        scheduler_data = df[df['scheduler'] == scheduler_name]
        metrics.append({
            'scheduler': scheduler_name,
            'mean_k': scheduler_data['k'].mean(),
            'std_k': scheduler_data['k'].std(),
            'k_range': scheduler_data['k'].max() - scheduler_data['k'].min(),
            'k_progression': scheduler_data['k'].iloc[-1] - scheduler_data['k'].iloc[0],
            'stability': 1 / (1 + scheduler_data['k'].std()),
            'coverage': len(scheduler_data['k'].unique()) / (K_MAX - K_MIN + 1)
        })
    return pd.DataFrame(metrics)

# Calcul des métriques
metrics_df = calculate_scheduler_metrics(base_df)

# Ajout du score global
metrics_df['score'] = (
    metrics_df['stability'] * 0.3 +
    metrics_df['coverage'] * 0.3 +
    metrics_df['k_progression'] * 0.4
)

# Affichage des métriques
metrics_df.sort_values('score', ascending=False)

## 5. Analyse de l'Influence d'Epsilon

In [ ]:
def analyze_epsilon_influence(best_scheduler):
    data = []
    for eps_config_name, eps_config in epsilon_configs.items():
        if isinstance(eps_config, (int, float)):
            scheduler = base_schedulers[best_scheduler].__class__(
                K_MIN, K_MAX, epsilon_max=eps_config
            )
        else:
            scheduler = CompositeScheduler(
                base_schedulers[best_scheduler].__class__(K_MIN, K_MAX),
                epsilon_type=eps_config,
                epsilon_min=0.1,
                epsilon_max=0.5
            )
        
        for epoch in range(MAX_EPOCHS):
            eps, k_dist = scheduler(epoch, MAX_EPOCHS)
            for k, prob in k_dist.items():
                data.append({
                    'epoch': epoch,
                    'epsilon_config': eps_config_name,
                    'k': k,
                    'epsilon': eps,
                    'probability': prob
                })
    return pd.DataFrame(data)

# Récupération du meilleur scheduler
best_scheduler = metrics_df.loc[metrics_df['score'].idxmax(), 'scheduler']
print(f"Meilleur scheduler identifié : {best_scheduler}")

# Analyse de l'influence d'epsilon
epsilon_df = analyze_epsilon_influence(best_scheduler)
epsilon_df.head()

## 6. Visualisation de l'Influence d'Epsilon

In [ ]:
plt.figure(figsize=(15, 10))
for eps_config in epsilon_configs.keys():
    config_data = epsilon_df[epsilon_df['epsilon_config'] == eps_config]
    plt.plot(config_data['epoch'], config_data['k'], 
            label=f'ε={eps_config}', alpha=0.7)

plt.title(f'Influence of Epsilon on {best_scheduler} Scheduler')
plt.xlabel('Epoch')
plt.ylabel('K Value')
plt.legend()
plt.grid(True)
plt.show()

## 7. Métriques d'Influence d'Epsilon

In [ ]:
epsilon_metrics = []
for eps_config in epsilon_configs.keys():
    config_data = epsilon_df[epsilon_df['epsilon_config'] == eps_config]
    epsilon_metrics.append({
        'epsilon_config': eps_config,
        'mean_k': config_data['k'].mean(),
        'std_k': config_data['k'].std(),
        'k_range': config_data['k'].max() - config_data['k'].min(),
        'mean_epsilon': config_data['epsilon'].mean()
    })

epsilon_metrics_df = pd.DataFrame(epsilon_metrics)
epsilon_metrics_df

## 8. Conclusion

À partir des analyses précédentes, nous pouvons conclure :
1. La meilleure stratégie de scheduler est [résultat]
2. L'influence d'epsilon sur cette stratégie est [significative/négligeable]
3. La meilleure configuration d'epsilon est [résultat]

Ces résultats suggèrent que [conclusion sur l'importance d'epsilon et les recommandations pour l'utilisation des schedulers]. 